# Calibrating a MultispeQ Device

This notebook is helping you to calibrate your MultispeQ device.

If you haven't used this notebook so far, make sure to install the following dependencies first:

```sh
pip install git+https://github.com/Jan-IngenHousz-Institute/JII-MultispeQ.git --upgrade --no-cache-dir
pip install git+https://github.com/Jan-IngenHousz-Institute/JII-MultispeQ-Protocols.git --upgrade --no-cache-dir
pip install matplotlib
```

In [ ]:
## The following dependencies are imported to use the MultispeQ and perform calibrations

import jii_multispeq.measurement as _measurement
import jii_multispeq.device as _device
import jii_multispeq.device.settings as _settings

from jii_multispeq_protocols import validate as _validate
from jii_multispeq_protocols.protocols import calibrations as _calibrations

import matplotlib.pyplot as plt
import json
import time

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

## PLEASE DO NOT CHANGE THIS CODE

## Your Protocol Code

MultispeQ Protocols are formatted as a JSON (JavaScript Object Notation). A pyton dictionary can be converted to a JSON, but it cannot contain functions as the MultispeQ is not able to interpret those.

### Protocol Variable

The next cell defines the variable `_protocol` which holds the protocol code. You can create the protocol simply writing the json code, but you can also use functions to build the code, as long as the final output for `_protocol` is a valid JSON.

### Protocol Validation

The cell after that validates the protocol, using the protocol validator function. This will help to identify issues with the protocol structure as well as with value ranges and parameters.

## Connect your MultispeQ Device

The first Step is to connect your MultispeQ Device to your computer and test the propper communication.

In [ ]:
## Identify the Port your MultispeQ instrument is connected to
_device.get_ports()

In [ ]:
## Enter the port name the device is connected to
_port = '/dev/cu.usbmodem42949672951'

## Connect your Instrument
_connection = _device.connect(_port)

In [ ]:
## Get the basic informatio about the connected device
if _device.is_connected(_connection):
  ## Send command
  data = _device.send_command(_connection, '1007')
  ## Display Data
  data = json.loads(data)
  _measurement.view(data)

## Save Current Device Settings

The current settings and Device Calibrations get saved here, so they can be restored later if needed.

In [ ]:
## Start the Device Calibration
if _device.is_connected(_connection):
  mem = _device.get_memory(_connection, True)
  filename = "%s_%s.json" % (data["device_id"], time.strftime("%Y-%m-%d") )

  with open(filename, "w") as file:
    json.dump(data, file)

  print('%sCurrent device settings saved here: %s%s' % (bcolors.OKGREEN, filename, bcolors.ENDC))

## Start the Device Calibration

Now it is time to start the actual Device calibration. Make sure you have the **CaliQ device**, the **Chlorophyll Calibration Cards**, the **Fluorescent Card (pink)**,  as well as the **Plastic Cards** with different thicknesses on hand.

### 1. Step - Check Battery Level

If your device Battery is **below 50%**, make sure to charge the MulispeQ device before you continue.

In [ ]:
if _device.is_connected(_connection):
  battery = _device.send_command(_connection, command="battery")

  try:
    battery = battery.strip().split(':')[1]
    if int(battery) < 50:
      print("%sBattery Level %s%%, please charge device before you continue%s" % (bcolors.FAIL, battery, bcolors.ENDC))
    else:  
      print('%sBattery Level of %s%% is sufficient%s' % (bcolors.OKGREEN, battery, bcolors.ENDC))
  except:
    print("%sData returned by command failed, found: %s%s" % (bcolors.FAIL, battery, bcolors.ENDC))


### 2. Step - Check CaliQ connection

Test the communication between the MultispeQ and the CaliQ device. If the test fails, make sure the connecting cable has the correct orientation.

In [ ]:
if _device.is_connected(_connection):
  _device.check_caliq_connection(_connection)

### 3. Step - Reset Instrument to Factory Settings

Reset the MultispeQ to its default settings. This requires a full recalibration to continue to use the MultispeQ.

In [ ]:
if _device.is_connected(_connection):
  data, crc32 = _measurement.measure(_connection, _calibrations.reset_to_default_settings._protocol)
  output = _measurement.analyze( data, _calibrations.reset_to_default_settings._analyze )
  _measurement.view(output)

### 4. Step - PAR Sensor Calibration

Before you begin, make sure the two halfs of the CaliQ are firmly attached to each other. Then, when prompted, carefully separate the two halfs. The side labeled with the PhotosynQ logo contains the LED. Take this side of the CaliQ and attach it to the MultispeQ's PAR sensor.

**Requirements:**
+ CaliQ

In [ ]:
if _device.is_connected(_connection):
  data, crc32 = _measurement.measure(_connection, _calibrations.par_sensor_calibration._protocol)
  output = _measurement.analyze( data, _calibrations.par_sensor_calibration._analyze )
  _measurement.view(output)

In [ ]:
## Now send the results back to the device
if "toDevice" in output and _device.is_connected(_connection):
  _device.send_command(_connection, command=output["toDevice"])
else:
  print("%sNo command found or device not connected.%s" % (bcolors.FAIL, bcolors.ENDC))


### 5. Step - Main Body LED Calibraton

*Remove the rubber seals around the MultispeQ's light guides before continuing.*

Attach the PAR sensor of the CaliQ (half without the Logo) around the light guide of the MultispeQ's main body. Adjust the CaliQ if needed before gently closing the leaf clamp.

**Requirements:**
+ Rubber seal removed
+ CaliQ PAR Sensor facing Main Body

In [ ]:
if _device.is_connected(_connection):
  data, crc32 = _measurement.measure(_connection, _calibrations.main_body_leds_calibration._protocol)
  output = _measurement.analyze( data, _calibrations.main_body_leds_calibration._analyze )
  _measurement.view(output)

### 6. Step - Leaf Clamp LED Calibraton

Attach the PAR sensor of the CaliQ (half without the Logo) around the light guide of the MultispeQ's Leaf Clamp. Adjust the CaliQ if needed before gently closing the leaf clamp.

**Requirements:**
+ Rubber seal removed
+ CaliQ PAR Sensor facing Leaf Clamp

In [ ]:
if _device.is_connected(_connection):
  data, crc32 = _device.measure(_connection, _calibrations.leaf_clamp_leds_calibration ._protocol)
  output = _measurement.analyze( data, _calibrations.leaf_clamp_leds_calibration._analyze )
  _measurement.view(output)

### 7. Step - Infrared LED Calibraton

*Insert the rubber seals back around the light guides before continuing.*

After starting the protocol, clamp the section(s) of the calibration cards as labeled and make sure to fully cover the light guide.

**Requirements:**
+ Rubber seal placed back
+ Chlorophyll Calibration Cards

In [ ]:
if _device.is_connected(_connection):
  data, crc32 = _device.measure(_connection, _calibrations.leaf_clamp_leds_calibration ._protocol)
  output = _measurement.analyze( data, _calibrations.leaf_clamp_leds_calibration._analyze )
  _measurement.view(output)

In [ ]:
# The calibration of the Infrared LEDs has failed. Please repeat the calibration.

### 8. Step - Set Detector Offsets

This step requires the white panels of the Chlorophyll calibration cards clamped as prompted covering the entire light guide.

**Requirements:**
+ Chlorophyll Calibration Cards

In [ ]:
if _device.is_connected(_connection):
  data, crc32 = _device.measure(_connection, _calibrations.electronic_offsets_calibration._protocol)
  output = _measurement.analyze( data, _calibrations.electronic_offsets_calibration._analyze )
  _measurement.view(output)

In [ ]:
# "The calibration of electronic offsets has failed. Please repeat the calibration."

### 8. Step - Set Detector Offsets (Fluorescence)

Take the pink fluorescence card and clamp it when prompted. Make sure to fully cover the light guide.

**Requirements:**
+ Fluorescent Card (pink)

In [ ]:
if _device.is_connected(_connection):
  data, crc32 = _device.measure(_connection, _calibrations.fluorescence_detector_offsets_calibration._protocol)
  output = _measurement.analyze( data, _calibrations.fluorescence_detector_offsets_calibration._analyze )
  _measurement.view(output)

In [ ]:
# "The calibration of electronic offsets has failed. Please repeat the calibration."

### 9. Step - Calibrate Leaf Thickness Gauge

Clamp the leaf thickness cards as prompted by the protocol and leave the dialog's input field blank. In case you use your own cards enter the values.

**Requirements:**
+ Thickness Calibration Cards

In [ ]:
if _device.is_connected(_connection):
  data, crc32 = _device.measure(_connection, _calibrations.leaf_thickness_gauge_calibration._protocol)
  output = _measurement.analyze( data, _calibrations.leaf_thickness_gauge_calibration._analyze )
  _measurement.view(output)

### 10. Step - Relative Chlorophyll Calibration

Clamp the chlorophyll calibration cards according tho their numbers when prompted with the MultispeQ's leaf clamp and enter the SPAD value from the cards envelope. Make sure, the cards cover the entire light guide. 

**Requirements:**
+ Chlorophyll Calibration Cards

In [ ]:
if _device.is_connected(_connection):
  data, crc32 = _device.measure(_connection, _calibrations.relative_chlorophyll_spad_calibration._protocol)
  output = _measurement.analyze( data, _calibrations.relative_chlorophyll_spad_calibration._analyze )
  _measurement.view(output)

In [ ]:
# "Relative Chlorophyll calibration has failed. Please repeat the calibration."

### 11. Step - Compass Calibration

To calibrate the devices internal compass, start moving the device in an elliptical motion until the calibration is finished.

Start moving the device immediately after starting the next step.

In [ ]:
if _device.is_connected(_connection):
  _device.calibrate_compass(_connection)

In [ ]:
# "Failed to properly calibrate the compass."

### 12. Step - Set Leaf Clamp **Open** Position

Open the Leaf Clamp about **4mm**, hold it in that position and run the next step.

In [ ]:
if _device.is_connected(_connection):
  _device.set_clamp_open(_connection)

In [ ]:
# "Failed to set the **Open** position."

### 13. Step - Set Leaf Clamp **Closed** Position

Close the Leaf Clamp to about **2mm**, hold it in that position and run the next step.

In [ ]:
if _device.is_connected(_connection):
  _device.set_clamp_closed(_connection)

In [ ]:
# "Failed to set the **Closed** position."